## Laboratorium 6

imports

In [1]:
import wikipedia
from wikipedia.exceptions import WikipediaException
import pickle
import csv
import numpy as np
import os
import random
import time
import re
from typing import List, Dict
import string
from gensim.parsing.preprocessing import remove_stopwords
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from collections import defaultdict
from scipy import sparse


[nltk_data] Downloading package stopwords to /home/hiro/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/hiro/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Tekst artukułu jest wyszukiwany i pobierany za pomocą biblioteki wikipedia. Po otrzymaniu danego artykułu jest on zapisywany do folderu ```articles``` za pomocą blbioteki ```pickle```.

Załadownie n-pierwszych artykułów z enwiki dump. (Jest to potrzebne ponieważ plik, który zainstalowałem ma ponad 300 000 tytułów artykułu)

In [2]:
class ArticleLoader:
    def __init__(self, n=10**4, file='articles/enwiki-pages-articles.txt'):
        self.n = n
        self.file = file
        self.enwiki_name = 'enwiki-pages-articles.txt'
        self.titles = []
        self.dict_of_key_words = dict()
    # zadanie 1 - przygotowanie zbioru dokumentów tekstowych
    def get_names_of_n_first_articles(self):
        with open(self.file, 'r',encoding="utf8") as f:
            csv_reader = csv.reader(f, delimiter=':')
            i = 0
            for row in csv_reader:
                if i >= self.n:
                    break

                title_of_article = row[-1]
            
                if '/' in title_of_article:
                    continue

                self.titles.append(title_of_article)
                i += 1

        random.shuffle(self.titles)
    
    def load_and_save_articles(self):
        not_found = 0
        if not self.titles:
            self.get_names_of_n_first_articles()

        for title in self.titles:
            try:
                wiki_page = wikipedia.page(title)
                url = wiki_page.url
                content = wiki_page.content
                dict_of_title = {'content': content, 'url': url}
                pickle.dump(dict_of_title, open('articles/{}'.format(title), 'wb'))
            except WikipediaException:
                not_found += 1
            except Exception as e:
                print("exception ocured {}".format(str(e)))
        print("Titles not found: {}".format(not_found))

    # zadabie 2 - określenie słownika słów kluczowych -> będzie to słwonik z kluczem jako nazwa artykułu, a wartością jako tekst artykułu
    
    # stworzenie nowego słownika dla słów kluczowych
    def create_dict_of_key_words(self, length = 1000):
        i = 0
        # self.dict_of_key_words["6"] = pickle.load(open("articles/{}".format(6), "rb"))

        for title in self.pickle_yeild():
            if i > length: break
            if title == self.enwiki_name or "key_words" in title: 
                continue
            
            self.dict_of_key_words[title] = pickle.load(open("articles/{}".format(title), "rb"))
            i+=1


        pickle.dump(self.dict_of_key_words, open('articles/dumped_data/dict_of_key_words_{}'.format(len(self.dict_of_key_words)), 'wb'))
    
    def pickle_yeild(self, path = 'articles'):
        for content in os.listdir(path):
            if os.path.isfile(os.path.join(path, content)):
                yield content

    # wczytanie wcześniej stworzonego słownika dla słów kluczowych
    def load_dict_of_key_words(self, dirpath = 'articles/dumped_data', dumped_file = 'dict_of_key_words_1001'):
        path = os.path.join(dirpath, dumped_file)
        self.dict_of_key_words = pickle.load(open(path, "rb"))




Stworzenie klasy ArticleLoader dla n elementów, pozwala ona na zarządzanie danymi z wikipedii. Można wczytać, zaspisać lub odczytać tytuły razem z linkami do tytułu.

In [3]:
article_handler = ArticleLoader(n= 1000)
article_handler.get_names_of_n_first_articles()


Wywołanie funkcji ```load_and_save_articles``` pozwala na przeszukanie wikipedii i zapisanie zawartości do pliku.

In [7]:
article_handler.load_and_save_articles()

/home/hiro/.local/lib/python3.10/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /home/hiro/.local/lib/python3.10/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


Titles not found: 125


Wywołanie funkcji ```create_dict_of_key_words``` pozwala na stworzenie słownika, w którym można odczytać zawartość pliku odnosząc się do tytułu artukułu.

In [13]:
article_handler.create_dict_of_key_words()

Wywołanie funkcji ```load_dict_of_key_words``` wczytuje do słownika wcześniej zapisany słownik -> pomocne gdyż tworzenie słownika zajmuje bardzo dużo czasu.

In [5]:
article_handler.load_dict_of_key_words()

### Klasa ```ArticleSearchEngine``` -> tworzenie programu wyszukującego artykuły

Inicjalizacja klasy ```ArticleSearchEngine``` przez słownik z kluczami jako tytuły oraz z wartościami jako słownik dla danego tytułu zawierającego wartość tekstową artykułu oraz link url

Klasa ta wykonuje kroki podane w treści zadań z laboratorium
- ```parse_article_text``` oraz ```article_parser```: parsowanie artykułu - zlicza ilości występowania danych wyrazów - jest to unia wszystkich wyrazów, które wystąpiły we wszystkuch artykułach oraz tworzy słownik z identyfikatorami, który jest pomocny w dalszej części klasy do zarządzania macierzami rzadkimi
- ```create_bags_of_words``` oraz ```fill_bag_of_words```: tworzenie *bag_of_words* - początkowo jest tworzona macierz rzadka o długości ilości wszystkich unikalnych słów. Następnie każda komurka jest wypełniana ilością wystąpienia słowa w danym artykule. Jest to operacja, która jest powtarzana dla każdego artykułu.
- ```create_term_by_document_matrix```: tworzona jest macierz *term-by-document matrix* w której wektory cech ułożone są kolumnowo (wektory -> odpowidni *bag_of_words*)
- ```multiply_by_IDF```: wstępne przetwarzanie otrzymanego zbioru elementy *bag_of_words* są mnożone przez *inverse document frequency*
- ```find_best_articles```: pozwala wprowadzać zapytania w postaci sekwencji słów, oraz zwraca *k* najbardziej trafnych dokumentów. Kożysta ona z funkcji ```parse_article_text```, by można było zawęzić zbiór przeszukiwać oraz dostosować wprowadzone zapytanie do przyjętej konwencji zapisywania słów. (np. goes zamieni się na go)
- ```normalize_vectors```: normalizuje wszystkie wektory w macierzy *term_by_document*.
- ```apply_SVD_and_LRA```: usuwa szum z macierzy *term_by_document* i jest jest stosowane low rank aproximation. Nowe miary prawdopodobieństwa są obliczane w funkcji ```find_best_articles```

In [53]:
class ArticleSearchEngine(object):
    def __init__(self, articles_key_words : Dict[str, Dict[str, str]]):
        self.articles = articles_key_words
        self.titles = list(self.articles.keys())
        self.occured_words = defaultdict(lambda: 0)
        self.uniq_occured_words = None
        self.ids_of_unique_occured_word = None
        self.parsed_article = dict()
        self.stop_words = set(stopwords.words('english'))
        self.lemmatisation = WordNetLemmatizer()
        self.term_by_document_matrix = None                 # macierz rzadka przechowująca wektory cech
        self.matrix_without_noise = None                    # przechowuje macierz po zastosowaniu SVD


    def parse_article_text(self, text: str, if_article = True):                     # dzięki if_article możemy sparsować dostosować parsowanie tekstu w zależności od czy jest artykułem czy nie
        parse_text = text.lower()                                                   # nie ma sensu w odróżnianiu małych i dużych liter
        parse_text = re.sub('['+string.punctuation+']', '', parse_text).split()     # parsowanie po tekście w celu usunięcia niepotrzebnych znajów np. #$%...
        parsed_text = [word for word in parse_text if word not in self.stop_words]  # usuwanie stop_wordów np. 'ours', 'i' ...

        words_dictonary = defaultdict(lambda: 0)                                    # dla nie powstałego słownika przypisz 0 jako wartość
        words_counter = 0                                                           # liczenie ilości słów
        for word in parsed_text:
            lemma_word = self.lemmatisation.lemmatize(word, pos='v')                # pos = 'v' - verb lemmatisation np. goes -> go ...
            words_dictonary[lemma_word] += 1                                        # dzięki defaultdict mamy pewność, że jak nie istnieje dany klucz to stworzy się nowy z wartością 0
            words_counter += 1
            if if_article:
                self.occured_words[lemma_word] += 1
        return dict(words_dictonary = words_dictonary, words_counter = words_counter)
            

    def article_parser(self):
        for article_title in self.titles:
            self.parsed_article[article_title] = dict()
            self.parsed_article[article_title]['article_content'] = self.parse_article_text(self.articles[article_title]['content'])

        self.uniq_occured_words = list(self.occured_words.keys())
        self.ids_of_unique_occured_word = {self.uniq_occured_words[i]: i for i in range(len(self.uniq_occured_words))}


    def create_bags_of_words(self):
        for article_title in self.titles:
            self.parsed_article[article_title]['bag_of_words'] = self.fill_bag_of_words(self.parsed_article[article_title]['article_content'])
    

    def fill_bag_of_words(self, article_content):
        article_words = article_content['words_dictonary'].keys()

        # tworzenie macierzy rzadkiej -> wymiary: ilość wszystkich unikalnych słów we wszystkich tekstów / 1 -> jeden, ponieważ działamy na jednym artykule w danym momencie
        # dok_matrix -> Dictionary Of Keys based sparse matrix, efficient structure for constructing sparse matrices incrementally
        vector = sparse.dok_matrix(np.zeros((len(self.ids_of_unique_occured_word), 1))) 
        for word_key in article_words:
            vector[self.ids_of_unique_occured_word[word_key], 0] = article_content['words_dictonary'][word_key]

        vector /= article_content['words_counter'] # normalizacja wektora -> dzielenie przez ilość unikalnych słów w artykule
        return sparse.csr_matrix(vector) # csr_matrix -> Compressed Sparse Row matrix
    
    
    def create_term_by_document_matrix(self):
        # n - ilość artukułów, m - ilość unikalnych słów w wszystkich tekstach
        n = len(self.titles)
        m = len(self.uniq_occured_words)
        self.term_by_document_matrix = sparse.lil_matrix((m, n)) # m jest liczbą termów w słowniku, n liczbą dokumentów
        for i in range(n):
            self.term_by_document_matrix[:,i] = self.parsed_article[self.titles[i]]['bag_of_words']
        # utworzenie skompresowanej macierzy rzadkiej by operacje arytmetyczny były w miarę czasowo wykonalane
        self.term_by_document_matrix = sparse.csr_matrix(self.term_by_document_matrix)


    def multiply_by_IDF(self):
        n = len(self.titles)
        for word in self.uniq_occured_words:
            # nw jest liczbą dokumentów, w których występuje słowo w
            nw = sum(1 for article in self.parsed_article.values() if word in article['article_content']['words_dictonary'])
            id_of_word = self.ids_of_unique_occured_word[word]
            idf_word = np.log(n/nw)
            self.term_by_document_matrix[id_of_word] *= idf_word
    

    def normalize_vector(self, vector):
        return np.sqrt((vector.power(2)).sum())


    def find_best_articles(self, input_words, num_of_articles_to_return = 10, with_noise = True):
        matrix = self.term_by_document_matrix if with_noise else self.matrix_without_noise

        parsed_input_words = self.parse_article_text(input_words, if_article = False)
        q = self.fill_bag_of_words(parsed_input_words)
        q_norm = self.normalize_vector(q)
        probs_of_article = [] # prawdopodobieństwo dla wystąpienia artykułu
        for i in range(len(self.titles)):
            d_i = matrix.getcol(i)
            d_i_norm = self.normalize_vector(d_i)
            cos_theta_i = (q.T @ d_i) / (q_norm * d_i_norm)
            probs_of_article.append((cos_theta_i, i)) # prawdopodowbieństwo oraz miejsce wystąpienia artykułu
        # return: search_link, tytuł, wartość prawdopodobieństwa
        probs_of_article.sort(key = lambda x: x[0], reverse = True)
        
        for prob, idx in probs_of_article[:num_of_articles_to_return]:
            found_article_title = self.titles[idx]
            found_article_url = self.articles[found_article_title]['url']
            print("url: {} \ttitle: {} \tprobability: {}".format(found_article_url, found_article_title, prob[(0,0)]))
    
    # normalizacja wszystkich wektorów wektorów
    def normalize_vectors(self):
        for i in range(len(self.titles)):
            vector = self.term_by_document_matrix.getcol(i)
            vector_norm = self.normalize_vector(vector)
            self.term_by_document_matrix[:, i] /= vector_norm


    def apply_SVD_and_LRA(self, k = 100):
        # zastosowanie svds ze względu na podanie k, a self.term_by_document_matrix jest macierzą rzadką więc chcemy wziąć k pierwszych wartości
        u, s, vh = sparse.linalg.svds(self.term_by_document_matrix, k=k)
        # utworzenie macierzy rzadkich z svd dla żądanego k
        u = sparse.csr_matrix(u)
        s = sparse.diags(s)
        vh = sparse.csr_matrix(vh)
        self.matrix_without_noise = u @ s @ vh # macierz po usunięciu szumów

    # stworzenie wielu metod do testowania następnych kroków wyszukiwarki
    def create_search_engine1(self):
        self.article_parser()
    def create_search_engine2(self):
        self.create_bags_of_words()
    def create_search_engine3(self):
        self.create_term_by_document_matrix()
    def create_search_engine4(self):
        self.multiply_by_IDF()
    def create_search_engine5(self):
        self.normalize_vectors()
    
    def create_search_engine(self):
        self.article_parser()
        self.create_bags_of_words()
        self.create_term_by_document_matrix()
        self.multiply_by_IDF()
        self.normalize_vectors()
        self.apply_SVD_and_LRA()


Stworzenie article_engine

In [54]:
article_engine = ArticleSearchEngine(article_handler.dict_of_key_words)
article_engine.create_search_engine()

Testowanie poprawnego wyszukiwania dla stworzonej macierzy bez szumów i różnych *k*

In [58]:
article_engine.apply_SVD_and_LRA(k = 50)
article_engine.find_best_articles("Colin Kendall Linden", num_of_articles_to_return = 5, with_noise = False) # dla k = 50

url: https://en.wikipedia.org/wiki/Mario_(singer) 	title: Mario (singer) 	probability: 0.03348503068017055
url: https://en.wikipedia.org/wiki/Colin_Linden 	title: Colin Linden 	probability: 0.033465387176103006
url: https://en.wikipedia.org/wiki/Sit_Down_Young_Stranger 	title: Sit Down Young Stranger 	probability: 0.032961571198391375
url: https://en.wikipedia.org/wiki/J._R._Richards 	title: J. R. Richards 	probability: 0.032944445395590646
url: https://en.wikipedia.org/wiki/Just_Another_Band_from_L.A. 	title: Just Another Band from L.A. 	probability: 0.032850163454966684


In [59]:
article_engine.apply_SVD_and_LRA(k = 100)
article_engine.find_best_articles("Colin Kendall Linden", num_of_articles_to_return = 5, with_noise = False) # dla k = 100

url: https://en.wikipedia.org/wiki/Colin_Linden 	title: Colin Linden 	probability: 0.04605549892111489
url: https://en.wikipedia.org/wiki/Yngwie_Malmsteen 	title: Fire &amp; Ice (Yngwie Malmsteen album) 	probability: 0.042881796033845995
url: https://en.wikipedia.org/wiki/Scott_Herren 	title: Crush the Sight-Seers 	probability: 0.04183507412712115
url: https://en.wikipedia.org/wiki/Still_Loving_You_(album) 	title: Still Loving You (album) 	probability: 0.04102087375003581
url: https://en.wikipedia.org/wiki/Time_Fades_Away 	title: Time Fades Away 	probability: 0.04077065172128933


In [60]:
article_engine.apply_SVD_and_LRA(k = 200)
article_engine.find_best_articles("Colin Kendall Linden", num_of_articles_to_return = 5, with_noise = False) # dla k = 200

url: https://en.wikipedia.org/wiki/Colin_Linden 	title: Colin Linden 	probability: 0.08190366909591817
url: https://en.wikipedia.org/wiki/Yngwie_Malmsteen 	title: Fire &amp; Ice (Yngwie Malmsteen album) 	probability: 0.05257579049509905
url: https://en.wikipedia.org/wiki/Still_Loving_You_(album) 	title: Still Loving You (album) 	probability: 0.04926353180979251
url: https://en.wikipedia.org/wiki/Mario_(singer) 	title: Mario (singer) 	probability: 0.04652137779220739
url: https://en.wikipedia.org/wiki/Scott_Herren 	title: Crush the Sight-Seers 	probability: 0.044149888528352936


In [61]:
article_engine.apply_SVD_and_LRA(k = 500)
article_engine.find_best_articles("Colin Kendall Linden", num_of_articles_to_return = 5, with_noise = False) # dla k = 500

url: https://en.wikipedia.org/wiki/Colin_Linden 	title: Colin Linden 	probability: 0.4775091707717041
url: https://en.wikipedia.org/wiki/Mick_Dalla-Vee 	title: Mick Dalla-Vee 	probability: 0.09394976555523996
url: https://en.wikipedia.org/wiki/Yngwie_Malmsteen 	title: Fire &amp; Ice (Yngwie Malmsteen album) 	probability: 0.08288007669545847
url: https://en.wikipedia.org/wiki/Sam_McDowell 	title: Sam McDowell 	probability: 0.05945335293903401
url: https://en.wikipedia.org/wiki/Time_Fades_Away 	title: Time Fades Away 	probability: 0.05828665638924148


In [62]:
article_engine.apply_SVD_and_LRA(k = 1000)
article_engine.find_best_articles("Colin Kendall Linden", num_of_articles_to_return = 5, with_noise = False) # dla k = 1000

url: https://en.wikipedia.org/wiki/Colin_Linden 	title: Colin Linden 	probability: 0.5762730715956088
url: https://en.wikipedia.org/wiki/Pink_Flag 	title: Pink Flag 	probability: 0.0375870832186858
url: https://en.wikipedia.org/wiki/Derek_and_Clive_Come_Again 	title: Derek and Clive Come Again 	probability: 0.03711340074574137
url: https://en.wikipedia.org/wiki/The_Two_of_Us_(1986_TV_series) 	title: The Two of Us (1986 TV series) 	probability: 0.029137283524072084
url: https://en.wikipedia.org/wiki/Alice_Mary_Robertson 	title: Alice Robertson 	probability: 0.02339126615783837


Testowanie poprawnego wyszukiwania dla macierzy podstawowej (zawiera szum)

In [56]:
article_engine.find_best_articles("Colin Kendall Linden", num_of_articles_to_return = 5)

url: https://en.wikipedia.org/wiki/Colin_Linden 	title: Colin Linden 	probability: 0.5762730715956087
url: https://en.wikipedia.org/wiki/Pink_Flag 	title: Pink Flag 	probability: 0.03758708321868575
url: https://en.wikipedia.org/wiki/Derek_and_Clive_Come_Again 	title: Derek and Clive Come Again 	probability: 0.03711340074574138
url: https://en.wikipedia.org/wiki/The_Two_of_Us_(1986_TV_series) 	title: The Two of Us (1986 TV series) 	probability: 0.02913728352407209
url: https://en.wikipedia.org/wiki/Alice_Mary_Robertson 	title: Alice Robertson 	probability: 0.023391266157838462


In [57]:
article_engine.find_best_articles("Colin Kendall Linden", num_of_articles_to_return = 10)


url: https://en.wikipedia.org/wiki/Colin_Linden 	title: Colin Linden 	probability: 0.5762730715956087
url: https://en.wikipedia.org/wiki/Pink_Flag 	title: Pink Flag 	probability: 0.03758708321868575
url: https://en.wikipedia.org/wiki/Derek_and_Clive_Come_Again 	title: Derek and Clive Come Again 	probability: 0.03711340074574138
url: https://en.wikipedia.org/wiki/The_Two_of_Us_(1986_TV_series) 	title: The Two of Us (1986 TV series) 	probability: 0.02913728352407209
url: https://en.wikipedia.org/wiki/Alice_Mary_Robertson 	title: Alice Robertson 	probability: 0.023391266157838462
url: https://en.wikipedia.org/wiki/Mick_Dalla-Vee 	title: Mick Dalla-Vee 	probability: 0.016986558357453708
url: https://en.wikipedia.org/wiki/Guadalupe_River_(Texas) 	title: Guadalupe River (Texas) 	probability: 0.01613152623085068
url: https://en.wikipedia.org/wiki/G-A-Y 	title: G-A-Y bar 	probability: 0.013995954788215961
url: https://en.wikipedia.org/wiki/Remain_in_Light 	title: Remain In Light 	probability: 

In [66]:
article_engine.find_best_articles("HDFN include pregnancies maternal circulation", num_of_articles_to_return = 10)

url: https://en.wikipedia.org/wiki/Hemolytic_disease_of_the_newborn 	title: Morbus haemolyticus neonatorum 	probability: 0.11289337112420099
url: https://en.wikipedia.org/wiki/Elders%27_Journal 	title: Elders' Journal 	probability: 0.028457036092927983
url: https://en.wikipedia.org/wiki/Playboy 	title: PlayBoy 	probability: 0.027959744081795898
url: https://en.wikipedia.org/wiki/Michalina_Wis%C5%82ocka 	title: Michalina Wisłocka 	probability: 0.026432470036111967
url: https://en.wikipedia.org/wiki/Stretch_marks 	title: Stretchmark 	probability: 0.023472727032756054
url: https://en.wikipedia.org/wiki/List_of_museum_ships 	title: Russian Museum.jpg 	probability: 0.019700374217848156
url: https://en.wikipedia.org/wiki/Bureau_of_Consular_Affairs 	title: Bureau of Consular Affairs 	probability: 0.019171731961135984
url: https://en.wikipedia.org/wiki/Valpara%C3%ADso_Region 	title: Región de Valparaíso 	probability: 0.018338555673390773
url: https://en.wikipedia.org/wiki/Djur%C3%B6_National_P

### Wnioski

- Wyszukiwarka znajduje poprawnie wyniki dla macierzy z szumami oraz bez
- najlepszy wynik (subiektywnie) wyszukiwarka daje z macierzą bez szumów dla *k* = 1000 - im mniejszy zakres tym słabsza miara prawdopodobieństwa dla danych wektorów.
- wyszukiwarka z szumami również podaje dobre wyniki, z szybszym działaniem